In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os
from collections import namedtuple
from math import sqrt
from statistics import mean


In [2]:
Point = namedtuple('Point', ['x', 'y'])


def length_between_points(a: Point, b: Point):
    squared = (pow(a.x - b.x, 2) + pow(a.y - b.y, 2))
    return sqrt(squared)

def normalize(raw):
    return [float(i)/max(raw) for i in raw]

def is_roughly_circle(x, y, confidence=0.1):
    center = Point(x=mean(x), y=mean(y))
    points = [Point(x[i], y[i]) for i in range(len(x))]

    lengths_from_center = [length_between_points(p, center) for p in points]
    normalized = normalize(lengths_from_center)
    is_circle = all([length > 1 - confidence for length in normalized])
    return is_circle

In [3]:
# Подключаем камеру

cap = cv2.VideoCapture(0) 
cap.set(3, 640) # Width
cap.set(4, 480) # Lenght
cap.set(10, 100) # Brightness


mpHands = mp.solutions.hands
hands = mpHands.Hands(False)
npDraw = mp.solutions.drawing_utils
a = []
pTime = 0
cTime = 0
data = []
tt = 0
cnt=0
ans = ''
while True:
    
    success, img = cap.read()
    img = cv2.flip(img,1) # Mirror flip
    cx8=0
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    pred2 = []
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            b = []
            for id, lm in enumerate(handLms.landmark):
                h,w,c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                b.append(lm.x)
                b.append(lm.y)
                if id == 8:
                    cx8 = cx
                if  id == 8 or id == 12:
                    cv2.circle(img, (cx,cy),10,(255,0,255),cv2.FILLED)
            
            npDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
            a. append(b)
    else:
         ans='nogest'
    
    if len(a)==20:
        coord=[]
        x=[]
        y=[]
        for i in a:
            coord.append(i[8])
        for i in range(len(coord)):
            if i==0 or i%2==0:
                x.append(coord[i])
            else:
                y.append(coord[i])
        print(is_roughly_circle(x,y))
        if is_roughly_circle(x,y,confidence=0.5)==True:
            print("circle")
        a=[]
    
    cv2.imshow('python', img)
    if cv2.waitKey(20) == 27: # exit on ESC
        break

cv2.destroyWindow("python")
cap.release()
cv2.waitKey(1)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
